In [1]:
import copy
import os.path as osp

import torch
import torch.nn.functional as F
from tqdm import tqdm

from torch_geometric.datasets import Reddit
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import SAGEConv

# download and loading the obg dataset
path = osp.join(osp.dirname(osp.realpath('./')), 'data', 'Reddit')
dataset = Reddit(path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device, 'x', 'y')

BS=2048
kwargs = {'batch_size': BS, 'num_workers': 6, 'persistent_workers': True}
train_loader = NeighborLoader(data, input_nodes=data.train_mask,
                              num_neighbors=[25, 10], shuffle=True, **kwargs)

subgraph_loader = NeighborLoader(copy.copy(data), input_nodes=None,
                                 num_neighbors=[-1], shuffle=False, **kwargs)
# No need to maintain these features during evaluation:
del subgraph_loader.data.x, subgraph_loader.data.y
# Add global node index information.
subgraph_loader.data.num_nodes = data.num_nodes
subgraph_loader.data.n_id = torch.arange(data.num_nodes)
# Already send node features/labels to GPU for faster access during sampling:

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [3]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  549652 KB |  549652 KB |  549652 KB |       0 B  |
|       from large pool |  549652 KB |  549652 KB |  549652 KB |       0 B  |
|       from small pool |       0 KB |       0 KB |       0 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |  549652 KB |  549652 KB |  549652 KB |       0 B  |
|       from large pool |  549652 KB |  549652 KB |  549652 KB |

In [7]:
LR=0.001
HC=8
NC=3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device, 'x', 'y')
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for i in range(NC-2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.convs) - 1:
                x = x.relu_()
                x = F.dropout(x, p=0.5, training=self.training)
            if i < len(self.convs) - 1 and i>0:
                x = conv(x, edge_index)
                x = x.relu_()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

    @torch.no_grad()
    def inference(self, x_all, subgraph_loader):
        print("testtest")
        pbar = tqdm(total=len(subgraph_loader.dataset) * len(self.convs))
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch:
        for i, conv in enumerate(self.convs):
            xs = []
            for batch in subgraph_loader:
                x = x_all[batch.n_id.to(x_all.device)].to(device)
                x = conv(x, batch.edge_index.to(device))
                if i < len(self.convs) - 1:
                    x = x.relu_()
                xs.append(x[:batch.batch_size].cpu())
                pbar.update(batch.batch_size)
            x_all = torch.cat(xs, dim=0)
            if i < len(self.convs) - 1 and i>0:
                xs = []
                for batch in subgraph_loader:
                    x = x_all[batch.n_id.to(x_all.device)].to(device)
                    x = conv(x, batch.edge_index.to(device))
                    x = x.relu_()
                    xs.append(x[:batch.batch_size].cpu())
                x_all = torch.cat(xs, dim=0)
        pbar.close()
        return x_all
    
model = SAGE(dataset.num_features, HC, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

def train(epoch):
    model.train()

    pbar = tqdm(total=int(len(train_loader.dataset)))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = total_examples = 0
    for batch in train_loader:
        optimizer.zero_grad()
        y = batch.y[:batch.batch_size]
        y_hat = model(batch.x, batch.edge_index.to(device))[:batch.batch_size]
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * batch.batch_size
        total_correct += int((y_hat.argmax(dim=-1) == y).sum())
        total_examples += batch.batch_size
        pbar.update(batch.batch_size)
    
    pbar.close()

    return total_loss / total_examples, total_correct / total_examples
@torch.no_grad()
def test():
    model.eval()
    y_hat = model.inference(data.x, subgraph_loader).argmax(dim=-1)
    y = data.y.to(y_hat.device)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((y_hat[mask] == y[mask]).sum()) / int(mask.sum()))
    return accs

In [8]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 3         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    1073 MB |    4387 MB |    4456 GB |    4455 GB |
|       from large pool |    1073 MB |    4386 MB |    4446 GB |    4445 GB |
|       from small pool |       0 MB |       2 MB |       9 GB |       9 GB |
|---------------------------------------------------------------------------|
| Active memory         |    1073 MB |    4387 MB |    4456 GB |    4455 GB |
|       from large pool |    1073 MB |    4386 MB |    4446 GB |

In [9]:
for epoch in range(1, 50):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
train_acc, val_acc, test_acc = test()
print(f'Epoch: {epoch:02d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
      f'Test: {test_acc:.4f}')

Epoch 01: 100%|██████████| 153431/153431 [00:04<00:00, 36769.70it/s]


Epoch 01, Loss: 3.5768, Approx. Train: 0.0613


Epoch 02: 100%|██████████| 153431/153431 [00:04<00:00, 37602.46it/s]


Epoch 02, Loss: 2.9223, Approx. Train: 0.1979


Epoch 03: 100%|██████████| 153431/153431 [00:04<00:00, 37794.63it/s]


...

Epoch 48, Loss: 0.7835, Approx. Train: 0.8258


Epoch 49: 100%|██████████| 153431/153431 [00:03<00:00, 38424.87it/s]


Epoch 49, Loss: 0.7726, Approx. Train: 0.8258
testtest


Evaluating: 100%|██████████| 698895/698895 [00:29<00:00, 23380.11it/s]


Epoch: 49, Train: 0.8395, Val: 0.8536, Test: 0.8526


In [10]:
print(torch.cuda.memory_summary())
for i, conv in enumerate(model.convs):
    print(conv)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 3         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    1073 MB |    4387 MB |   11728 GB |   11727 GB |
|       from large pool |    1073 MB |    4386 MB |   11703 GB |   11702 GB |
|       from small pool |       0 MB |       2 MB |      25 GB |      25 GB |
|---------------------------------------------------------------------------|
| Active memory         |    1073 MB |    4387 MB |   11728 GB |   11727 GB |
|       from large pool |    1073 MB |    4386 MB |   11703 GB |

In [16]:
len(model.convs)

3

0
